In [13]:
import pandas as pd

In [8]:
case = "3-bus"
nb_stages = 10
folder = "D.Representative_days_based_on_RES_and_Demand"

In [9]:
df_investment_results = pd.read_csv(f"../{folder}/{case}_ByStages_nc{nb_stages}/3.Out/oT_Result_NetworkInvestment_{case}_ByStages_nc{nb_stages}.csv")

In [10]:
# Specify the positions of the columns to be used as the index (adjust as needed)
index_columns = [0, 1, 2]

# Read the CSV file and set the specified columns as the index
df_parameters = pd.read_csv(f"{case}/2.Par/oT_Data_Network_{case}.csv", index_col=index_columns)

# Specify the names for the index columns
index_names = ['InitialNode', 'FinalNode', 'Circuit']

# Set the names for the index columns
df_parameters.index.set_names(index_names, inplace=True)
df_investment_results.set_index(index_names,inplace=True)
df_investment_results["Sensitivity"] = True

In [11]:
df_parameters["InvestmentFixed"] = df_investment_results["p.u."]
df_parameters["Sensitivity"] = df_investment_results["Sensitivity"]

In [24]:
df_parameters.index.names = [None] * len(df_parameters.index.names)
df_parameters.to_csv(f"{case}/2.Par/oT_Data_Network_{case}.csv")

In [36]:
#Test loop with copying

import os
import re

min_nb = 10
max_nb = 10

path_to_scan = os.path.dirname("../D.Representative_days_based_on_RES_and_Demand/")

filter = f'{case}_ByStages_nc'

directories = []

for f in os.scandir(path_to_scan):
    if f.is_dir():
        basename = os.path.basename(f.path)
        match = re.search(f'{filter}(\d+)', basename)
        if match:
            nb = int(match.group(1))
            if (min_nb is None or nb >= min_nb) and (max_nb is None or nb <= max_nb):
                directories.append(basename)

directories = sorted(directories, key=lambda x: int(re.search(r'nc(\d+)', x).group(1)))

print(directories)

['3-bus_ByStages_nc10']


In [5]:
#Test script without re-building model
import oSN_Main_operational
import pyomo

openStarNet = pyomo.environ.ConcreteModel('openStarNet - Open Version of the StartNetLite Model (Long Term Transmission Expansion Planning) - Version 1.0.0 - January 16, 2023')
openStarNet.pLineXNetInv = 0

oSN_Main_operational.data_processing("","3-bus",openStarNet)
oSN_Main_operational.create_variables(openStarNet,openStarNet)
oSN_Main_operational.create_constraints(openStarNet,openStarNet)



 #### Academic research license - for non-commercial use only #### 

Reading    input data                 ...  0 s
Setting up input data                 ...  6 s
Generating objective function         ...  4 s
Generating operation & investment     ...  0 s
Generating balance                    ...  6 s
Generating storage operation          ...  0 s
Generating generation commitment      ...  2 s
Generating generation capacity        ...  1 s
Generating network    capacity        ...  7 s
Generating network    constraints     ...  19 s
Econometric analysis  constraints     ...  0 s


In [15]:
#Read investment decisions
folder = "D.Representative_days_based_on_RES_and_Demand"
case = "3-bus"
nb_stages = 10
origin_folder = f"../{folder}/{case}_ByStages_nc{nb_stages}/3.Out/oT_Result_NetworkInvestment_{case}_ByStages_nc{nb_stages}.csv"
df_investment_results = pd.read_csv(origin_folder)
df_investment_results

,Period,InitialNode,FinalNode,Circuit,p.u.
0,2030,Node_1,Node_3,cac3,-0.0
1,2030,Node_1,Node_2,cac1,-0.0
2,2030,Node_2,Node_3,cac2,-0.0
3,2030,Node_2,Node_3,cac3,-0.0
4,2030,Node_1,Node_3,cac1,-0.0
5,2030,Node_1,Node_2,cac2,-0.0
6,2030,Node_1,Node_2,cac3,-0.0
7,2030,Node_2,Node_3,cac1,1.0
8,2030,Node_1,Node_3,cac2,-0.0


In [41]:
df_investment_results

,Period,InitialNode,FinalNode,Circuit,p.u.
0,2030,Node_1,Node_3,cac3,-0.0
1,2030,Node_1,Node_2,cac1,-0.0
2,2030,Node_2,Node_3,cac2,-0.0
3,2030,Node_2,Node_3,cac3,-0.0
4,2030,Node_1,Node_3,cac1,-0.0
5,2030,Node_1,Node_2,cac2,-0.0
6,2030,Node_1,Node_2,cac3,-0.0
7,2030,Node_2,Node_3,cac1,1.0
8,2030,Node_1,Node_3,cac2,-0.0


In [46]:
 #%% fix values of binary variables to get dual variables and solve it again
print('# ============================================================================= #')
print('# ============================================================================= #')
idx = 0
for var in openStarNet.component_data_objects(pyomo.environ.Var, active=True, descend_into=True):
    if not var.is_continuous():
        node_from,node_to,cac = var.index()[1],var.index()[2],var.index()[3]
        value_to_fix = find_pu_value_from_inv_results_df(df_investment_results,node_from=node_from,node_to=node_to,cac=cac)
        #var.fixed = True  # fix the current value
        print("fixing: " + str(var) + "to " + str(value_to_fix))
        var.fix(value=value_to_fix)
        idx += 1
print("Number of fixed variables: ", idx)
print('# ============================================================================= #')
print('# ============================================================================= #')

# ============================================================================= #
# ============================================================================= #
fixing: vNetworkInvest[2030,Node_1,Node_2,cac1]to -0.0
fixing: vNetworkInvest[2030,Node_2,Node_3,cac1]to 1.0
fixing: vNetworkInvest[2030,Node_1,Node_2,cac3]to -0.0
fixing: vNetworkInvest[2030,Node_1,Node_2,cac2]to -0.0
fixing: vNetworkInvest[2030,Node_2,Node_3,cac3]to -0.0
fixing: vNetworkInvest[2030,Node_2,Node_3,cac2]to -0.0
fixing: vNetworkInvest[2030,Node_1,Node_3,cac1]to -0.0
fixing: vNetworkInvest[2030,Node_1,Node_3,cac3]to -0.0
fixing: vNetworkInvest[2030,Node_1,Node_3,cac2]to -0.0
Number of fixed variables:  9
# ============================================================================= #
# ============================================================================= #
